# Logistic Regression

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# For our modeling steps
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import log_loss

# For demonstrative pruposes
from scipy.special import logit, expit

## Objectives

- Describe conceptually the need to move beyond linear regression
- Explain the form of logistic regression

[Wikipedia](https://en.wikipedia.org/wiki/Generalized_linear_model) has a nice description of the need to move beyond linear regression for certain sorts of modeling problems.

## Classification

**Classification techniques** are an essential part of machine learning and data mining applications. Most problems in Data Science are classification problems. 

The target is now a **categorical variable** instead of a numerical variable.

![classification](https://static.javatpoint.com/tutorial/machine-learning/images/regression-vs-classification-in-machine-learning.png)

Image source: [JavaTPoint](https://www.javatpoint.com/regression-vs-classification-in-machine-learning)

 So classification is the right paradigm for:

- image classification
- churn prediction
- species identification
- medical diagnosis
- "Which ___ are You?"

There are lots of classification algorithms that are available, but we'll focus here on logistic regression.

We shall focus on binary classification problems, to which logistic regression most immediately applies. Other classification problems handle the cases where multiple classes are present in the target variable.

## Predicting a Categorical Response

Here we have a dataset about glass. Information [here](https://archive.ics.uci.edu/ml/datasets/glass+identification).

## Preparing Data

In [ ]:
# glass identification dataset
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/glass/glass.data'
col_names = ['id','ri','na','mg','al','si','k','ca','ba','fe','glass_type']
glass = pd.read_csv(url, names=col_names, index_col='id')
glass.sort_values('al', inplace=True)
glass.head()

In [ ]:
# types 1, 2, 3 are window glass
# types 5, 6, 7 are household glass
glass['household'] = glass['glass_type'].map({1:0, 2:0, 3:0, 5:1, 6:1, 7:1})
glass.head()

Let's change our task, so that we're predicting **household** using **al**. Let's visualize the relationship to figure out how to do this:

In [ ]:
fig, ax = plt.subplots()
ax.scatter(glass['al'], glass['household'])
ax.set_xlabel('al')
ax.set_ylabel('household')
ax.set_title('Type of Glass as a Function of Aluminum Content');

## Using a Regression Line

Let's draw a **regression line**, like we did before.

**Note**: In the context of machine learning you'll want always to do a train-test split before modeling, but we'll forgo that here for the sake of convenience.

In [ ]:
# fit a linear regression model and store the predictions

linreg = LinearRegression()
feature_cols = ['al']
X = glass[feature_cols]
y = glass['household']
linreg.fit(X, y)
glass['household_pred'] = linreg.predict(X)

In [ ]:
# scatter plot that includes the regression line

fig, ax = plt.subplots()
ax.scatter(glass['al'], glass['household'])
ax.plot(glass['al'], glass['household_pred'], color='red')
ax.set_xlabel('al')
ax.set_ylabel('household');

> What are some issues with this graph?

## Interpreting Our Predictions

If **al=3**, what class do we predict for household? 

If **al=1.5**, what class do we predict for household? 

We predict the 0 class for **lower** values of al, and the 1 class for **higher** values of al. What's our cutoff value? Around **al=2**, because that's where the linear regression line crosses the midpoint between predicting class 0 and class 1.

Therefore, we'll say that if **household_pred $\geq$ 0.5**, we predict a class of **1**, else we predict a class of **0**.

# Logistic Regression

Logistic regression can do what we just did.

The strategy now is to *generalize* the notion of linear regression; linear regression as we've known it will become a special case. In particular, we'll keep the idea of the regression best-fit line, but now **we'll allow the model to make predictions through some (non-trivial) transformation of the linear predictor**.

Let's say we've constructed our best-fit line, i.e. our linear predictor, $\hat{L} = \beta_0 + \beta_1x_1 + ... + \beta_nx_n$.

Consider the following transformation: <br/>
$\large\hat{y} = \Large\frac{1}{1 + e^{-\hat{L}}} \large= \Large\frac{1}{1 + e^{-(\beta_0 + ... + \beta_nx_n)}}$. This is called the **sigmoid function**.

We're imagining that $\hat{L}$ can take any values between $-\infty$ and $\infty$.

$\large\rightarrow$ But what values can $\hat{y}$ take? What does this function even look like?

In `scipy` this function is called the ***expit*** function.

In [ ]:
# Let's plot this function here:

X = np.linspace(-10, 10, 300)
Y = expit(X)

fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(X, Y, 'r');

## Interpretation

This function squeezes our predictions between 0 and 1. And that's why it's so useful for **binary classification problems**.

Suppose I'm building a model to predict whether a plant is poisonous or not, based perhaps on certain biological features of its leaves. I'll let '1' indicate a poisonous plant and '0' indicate a non-poisonous plant.

Now I'm forcing my predictions to be between 0 and 1, so suppose that for test plant $P$ I get some value like 0.19.

I can naturally understand this as **the probability that $P$ is poisonous**.

If I truly want a binary prediction, I can simply round my score appropriately.

## Fitting Logistic Regression

### The Logit Function

We just expressed the form of logistic regression in terms of the sigmoid function: **Our model's predictions ($\hat{y}$) are not now identical with the values of the best-fit line but rather with the outputs of the sigmoid function, with those best-fit values passed as input.**

But we can also describe the best-fit line as a function of $\hat{y}$, by applying the **inverse of the sigmoid function** to both sides. This inverse function is called the ***logit* function**:

$ln(\frac{y}{1-y}) = \hat{L} = \beta_0+\beta_1x_1 +...+\beta_nx_n$.

<details>
    <summary>Proof that logit is the inverse of expit</summary>
Start with expit and then swap $x$ and $y$. Let $x = \frac{1}{1 + e^{-y}}$; <br/>
then $1 + e^{-y} = \frac{1}{x}$; <br/>
so $e^{-y} = \frac{1 - x}{x}$; <br/>
so $-y = \ln\left(\frac{1 - x}{x}\right)$; <br/>
so $y = \ln\left(\frac{x}{1 - x}\right)$
    </details>

In [ ]:
# Let's plot the logit function here:

X = np.linspace(0, 1, 300)
Y = logit(X)

fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(X, Y, 'r');

This fraction, $\frac{y}{1-y}$, is the **odds ratio** of y.

### Odds

There are other ways to squeeze the results of a linear regression into the set (0, 1).

But the ratio $\frac{p}{1-p}$ represents the *odds* of some event, where $p$ is the probability of the event.

$$probability = \frac {one\ outcome} {all\ outcomes}$$

$$odds = \frac {one\ outcome} {all\ other\ outcomes}$$

Examples:

- Dice roll of 1: probability = 1/6, odds = 1/5
- Even dice roll: probability = 3/6, odds = 3/3 = 1
- Dice roll less than 5: probability = 4/6, odds = 4/2 = 2

$$odds = \frac {probability} {1 - probability}$$

$$probability = \frac {odds} {1 + odds}$$

And so the logit function represents the **log-odds** of success (y=1).

Let's try applying the logit function to our target and then fitting a linear regression to that. Since the model will be trained not on whether the glass is household but rather on *the logit of this label*, it will also make predictions of the logit of that label. But we can simply apply the sigmoid function to the model's output to get its predictions of whether the glass is household.

We can't use the target as is, because the logit of 1 is $\infty$ and the logit of 0 is $-\infty$.

In [ ]:
glass['household'].unique()

In [ ]:
logit(glass['household'].unique())

So we'll make a small adjustment:

In [ ]:
target_approx = np.where(glass['household'] == 0, 1e-9, 1-1e-9)

In [ ]:
line_to_logit = LinearRegression()

X = glass[['al']]
y = logit(target_approx)

line_to_logit.fit(X, y)

Now we'll throw the predictions from the linear regression into the sigmoid function to get our class predictions!

In [ ]:
fig, ax = plt.subplots()

final_preds = expit(line_to_logit.predict(X))
ax.scatter(X, glass['household'])
ax.plot(X, final_preds, 'm');

## `sklearn.linear_model.LogisticRegression()`

In general, we should always scale our data when using this class. Scaling is always important for models that include regularization, and scikit-learn's `LogisticRegression()` objects have regularization by default.

Here we've forgone the scaling since we only have a single predictor.

In [ ]:
# fit a logistic regression model and store the class predictions

logreg = LogisticRegression(random_state=42)
feature_cols = ['al']
X = glass[feature_cols]
y = glass['household']
logreg.fit(X, y)
glass['household_pred_class'] = logreg.predict(X)

In [ ]:
# plot the class predictions

fig, ax = plt.subplots()
ax.scatter(glass['al'], glass['household'])
ax.plot(glass['al'], glass['household_pred_class'], color='red')
ax.set_xlabel('al')
ax.set_ylabel('household');

## `.predict()` vs. `.predict_proba()`

Let's checkout some specific examples to make predictions with. We'll use both `predict()` and `predict_proba()`.

In [ ]:
glass['al']

In [ ]:
# examine some example predictions

print("Here's what the .predict() method returns:")
print(logreg.predict(glass['al'][22].reshape(1, -1)))
print(logreg.predict(glass['al'][185].reshape(1, -1)))
print(logreg.predict(glass['al'][164].reshape(1, -1)))
print('\n')
print("Here's what the .predict_proba() method returns:")
print(logreg.predict_proba(glass['al'][22].reshape(1, -1))[0])
print(logreg.predict_proba(glass['al'][185].reshape(1, -1))[0])
print(logreg.predict_proba(glass['al'][164].reshape(1, -1))[0])
first_row = glass['al'][22].reshape(1, -1)

The first column indicates the predicted probability of **class 0**, and the second column indicates the predicted probability of **class 1**.

In [ ]:
# store the predicted probabilites of class 1
glass['household_pred_prob'] = logreg.predict_proba(X)[:, 1]

In [ ]:
# plot the predicted probabilities
fig, ax = plt.subplots()
ax.scatter(glass['al'], glass['household'])
ax.plot(glass['al'], glass['household_pred_prob'], color='red')
ax.set_xlabel('al')
ax.set_ylabel('household');

## Level Up: Measuring the Quality of a Classification Model

When we were constructing regression models we often used metrics like $R^2$ and $MSE$. How do we measure the quality of a classification model?

One metric we can use is **log-loss**:

$\mathcal{L}(\vec{y}, \hat{\vec{y}}) = -\frac{1}{N}\Sigma^N_{i=1}\left(y_iln(\hat{y}_i)+(1-y_i)ln(1-\hat{y}_i)\right)$,

where $\hat{y}_i$ is the probability that $(x_{i1}, ... , x_{in})$ belongs to **class 1**.

How can we think of the baseline score for this metric?

Suppose our target looks like this:

In [ ]:
truth = [0, 0, 1, 1]

A natural choice for a baseline model would simply predict $50\%$ every time:

In [ ]:
preds = [0.5, 0.5, 0.5, 0.5]

Let's calculate the log loss for such a model:

In [ ]:
log_loss(truth, preds)

Notice that is simply equal to $-ln(0.5)$, or, equivalently, $ln(2)$:

In [ ]:
-np.log(0.5)

In [ ]:
np.log(2)

Notice also that this model is *not* equivalent to a model that randomly makes predictions of $1$s and $0$s:

In [ ]:
preds2 = [0, 1, 1, 0]

In [ ]:
log_loss(truth, preds2)

The `log_loss()` function uses a default $\epsilon$ value of $1e-15$.

And so the relevant calculation here is:

In [ ]:
-0.5*(np.log(1-1e-15) + np.log(1e-15))

If we adjust the $\epsilon$ value to 0.5, we get back our baseline log-loss score:

In [ ]:
log_loss(truth, preds2, eps=0.5)

For *our* model, the baseline score really depends on the distribution of our target. Let's meaasure this:

In [ ]:
dist = glass['household'].sum() / glass['household'].count()
dist

So our baseline log-loss score would really be what we get if we used this value as a prediction for *all* inputs:

In [ ]:
preds_base = [dist] * len(X)

In [ ]:
log_loss(glass['household'], preds_base)

With that baseline in mind, let's check out the log-loss on our glass model:

In [ ]:
log_loss(glass['household'], logreg.predict_proba(X)[:, 1])

Not bad!

## Level Up: Interpreting Logistic Regression Coefficients

In [ ]:
logreg.coef_

How do we interpret the coefficients of a logistic regression? For a linear regression, the situaton was like this:

- Linear Regression: We construct the best-fit line and get a set of coefficients. Suppose $\beta_1 = k$. In that case we would expect a 1-unit change in $x_1$ to produce a $k$-unit change in $y$.

Now for logistic regression we have:

- Logistic Regression: We find the coefficients of the best-fit line by some approximation method. Suppose $\beta_1 = k$. In that case we would expect a 1-unit change in $x_1$ to produce a $k$-unit change (not in $y$ but) in $ln\left(\frac{y}{1-y}\right)$.

<details>
    <summary>Proof</summary>
We have:

$\ln\left(\frac{y(x_1+1, ... , x_n)}{1-y(x_1+1, ... , x_n)}\right) = \ln\left(\frac{y(x_1, ... , x_n)}{1-y(x_1, ... , x_n)}\right) + k$.

Exponentiating both sides:

$\frac{y(x_1+1, ... , x_n)}{1-y(x_1+1, ... , x_n)} = e^{\ln\left(\frac{y(x_1, ... , x_n)}{1-y(x_1, ... , x_n)}\right) + k}$ <br/><br/> $\frac{y(x_1+1, ... , x_n)}{1-y(x_1+1, ... , x_n)}= e^{\ln\left(\frac{y(x_1, ... , x_n)}{1-y(x_1, ... , x_n)}\right)}\cdot e^k$ <br/><br/> $\frac{y(x_1+1, ... , x_n)}{1-y(x_1+1, ... , x_n)}= e^k\cdot\frac{y(x_1, ... , x_n)}{1-y(x_1, ... , x_n)}$

That is, the odds ratio at $x_1+1$ has increased by a factor of $e^k$ relative to the odds ratio at $x_1$.
</details>

For more on this interpretation, see [this page](https://support.minitab.com/en-us/minitab-express/1/help-and-how-to/modeling-statistics/regression/how-to/binary-logistic-regression/interpret-the-results/all-statistics-and-graphs/coefficients/).

In [ ]:
# examine the intercept

logreg.intercept_

**Interpretation:** For an 'al' value of 0, the log-odds of 'household' is -6.01. What is the probability that glass with an 'al' value of 0 is household glass?

In [ ]:
# convert log-odds to probability

logodds = logreg.intercept_
prob = expit(logodds)
prob

In [ ]:
# examine the coefficient for al

list(zip(feature_cols, logreg.coef_[0]))

**Interpretation:** A 1 unit increase in 'al' is associated with a 3.12-unit increase in the log-odds of 'household'.

##### Aside: Verifying log-odds to probability

Let's verify this as we change the aluminum content from 1 to 2.

In [ ]:
# Prediction for al=1

pred_al1 = logreg.predict_proba([[1]])
pred_al1

In [ ]:
# Odds ratio for al=1

odds_al1 = pred_al1[0][1] / pred_al1[0][0]
odds_al1

In [ ]:
# Prediction for al=2

pred_al2 = logreg.predict_proba([[2]])
pred_al2

In [ ]:
# Odds ratio for al=2

odds_al2 = pred_al2[0][1] / pred_al2[0][0]
odds_al2

print((np.exp(logreg.coef_[0]) * odds_al1)[0])
print(odds_al2)

##### Aside: Use Coefficients to Generate Prediction

In [ ]:
# compute predicted log-odds for al=2 using the equation

x_al = 2
logodds = logreg.intercept_ + logreg.coef_[0] * x_al
logodds

In [ ]:
# convert log-odds to probability

prob = expit(logodds)
prob

In [ ]:
# compute predicted probability for al=2 using the predict_proba method

logreg.predict_proba(np.array([2]).reshape(1, 1))[:, 1]